In [26]:
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import asyncio
import json
from agentfield import AIConfig, Agent

In [27]:
import os

app = Agent(
    node_id="simulation-engine",
    agentfield_server=f"{os.getenv('AGENTFIELD_SERVER', 'http://localhost:8080')}",
    ai_config=AIConfig(
        model=os.getenv("AI_MODEL", "openrouter/deepseek/deepseek-v3.1-terminus"),
        api_key="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",
    ),
)

In [28]:
class ScenarioAnalysis(BaseModel):
    """Simple schema for scenario decomposition"""

    entity_type: str = Field(
        description="Type of entity being simulated (e.g., 'customer', 'voter', 'employee')"
    )
    decision_type: str = Field(
        description="Type of decision (e.g., 'binary_choice', 'multi_option', 'continuous_value')"
    )
    decision_options: List[str] = Field(
        description="List of possible decisions/outcomes"
    )
    analysis: str = Field(
        description="Detailed analysis of the scenario including key factors, causal relationships, and what matters"
    )


async def decompose_scenario(
    scenario: str, context: List[str] = []
) -> ScenarioAnalysis:
    """
    Analyzes the scenario to understand what we're simulating.
    Returns entity type, decision type, and deep analysis.
    """
    context_str = (
        "\n".join([f"- {c}" for c in context])
        if context
        else "No additional context provided."
    )

    prompt = f"""You are analyzing a simulation scenario to understand what needs to be modeled.

SCENARIO:
{scenario}

CONTEXT:
{context_str}

TASK:
Analyze this scenario deeply and provide:

1. entity_type: What type of entity/person are we simulating? (e.g., "customer", "voter", "employee", "consumer")

2. decision_type: What kind of decision are they making?
   - "binary_choice" (yes/no, stay/leave)
   - "multi_option" (choose from several options)
   - "continuous_value" (a number or amount)

3. decision_options: List all possible decisions/outcomes the entity could make. Be specific and exhaustive.

4. analysis: Write a comprehensive analysis (3-4 paragraphs) covering:
   - What are the key factors that would influence this decision?
   - What causal relationships exist? (e.g., "income affects price sensitivity")
   - What attributes of the entity would matter most?
   - What psychological, economic, or social dynamics are at play?
   - Are there different segments/archetypes of entities we should consider?
   - What hidden variables or second-order effects might exist?

Be thorough in your analysis - this will guide the entire simulation."""

    result = await app.ai(prompt, schema=ScenarioAnalysis)

    return result

In [29]:
class FactorGraph(BaseModel):
    """Schema for the causal attribute graph"""

    attributes: Dict[str, str] = Field(
        description="Dictionary of attribute_name: description. Each attribute that matters for this entity."
    )
    attribute_graph: str = Field(
        description="Detailed description of how attributes relate to each other and to the decision, including correlations, dependencies, and causal chains"
    )
    sampling_strategy: str = Field(
        description="Description of how to sample these attributes to get realistic, diverse entities"
    )


async def generate_factor_graph(
    scenario: str, scenario_analysis: ScenarioAnalysis, context: List[str] = []
) -> FactorGraph:
    """
    Creates the factor graph: what attributes matter and how they relate.
    """
    context_str = (
        "\n".join([f"- {c}" for c in context])
        if context
        else "No additional context provided."
    )

    prompt = f"""You are designing the factor graph for a simulation.

SCENARIO:
{scenario}

CONTEXT:
{context_str}

PREVIOUS ANALYSIS:
Entity Type: {scenario_analysis.entity_type}
Decision Type: {scenario_analysis.decision_type}
Possible Decisions: {', '.join(scenario_analysis.decision_options)}

Key Insights from Analysis:
{scenario_analysis.analysis}

TASK:
Design the factor graph that defines what attributes each {scenario_analysis.entity_type} should have.

1. attributes: Create a dictionary of all relevant attributes. For each attribute, provide a clear description.
   Include attributes across these categories:
   - Demographic (age, location, income, etc.)
   - Behavioral (usage patterns, preferences, history)
   - Psychographic (values, attitudes, personality traits)
   - Contextual (external factors, constraints, alternatives available)

   Keep attribute names simple and lowercase (e.g., "age", "income_level", "price_sensitivity")
   Make descriptions clear and specific.

2. attribute_graph: Write a detailed explanation (2-3 paragraphs) of:
   - How attributes influence each other (correlations and dependencies)
   - How attributes influence the final decision
   - What are strong vs weak predictors
   - Any interaction effects (e.g., "age matters more for low-income entities")
   - Which attributes cluster together to form natural segments

3. sampling_strategy: Describe how to sample these attributes to create realistic entities:
   - What are typical ranges/distributions for each attribute?
   - Which attributes are correlated and should be sampled together?
   - Are there natural segments/archetypes we should ensure are represented?
   - What makes a "realistic" vs "unrealistic" combination of attributes?

Be specific and detailed - this defines the entire simulation space."""

    result = await app.ai(prompt, schema=FactorGraph)

    return result

In [30]:
class EntityProfile(BaseModel):
    """Schema for a single entity's attributes"""

    entity_id: str
    attributes: Dict[str, Any] = Field(
        description="Dictionary of attribute_name: value for this entity"
    )
    profile_summary: str = Field(
        description="2-3 sentence human-readable summary of who this entity is"
    )


async def generate_entity_profile(
    entity_id: str,
    scenario_analysis: ScenarioAnalysis,
    factor_graph: FactorGraph,
    exploration_mode: bool = False,
) -> EntityProfile:
    """
    Generates a single entity with realistic attributes.

    Args:
        exploration_mode: If True, sample from edges/unusual combinations.
                         If False, sample from typical distributions.
    """
    mode_instruction = ""
    if exploration_mode:
        mode_instruction = """
EXPLORATION MODE: Generate an entity with unusual or edge-case attributes.
Sample from the tails of distributions or create surprising but realistic combinations.
This helps us discover unexpected scenarios."""
    else:
        mode_instruction = """
STANDARD MODE: Generate a typical, realistic entity following normal distributions
and common attribute combinations."""

    prompt = f"""You are generating a synthetic {scenario_analysis.entity_type} for simulation.

ENTITY TYPE: {scenario_analysis.entity_type}

AVAILABLE ATTRIBUTES:
{json.dumps(factor_graph.attributes, indent=2)}

ATTRIBUTE RELATIONSHIPS:
{factor_graph.attribute_graph}

SAMPLING GUIDANCE:
{factor_graph.sampling_strategy}

{mode_instruction}

TASK:
Generate a realistic {scenario_analysis.entity_type} by:

1. attributes: For each attribute in the list above, assign a specific value.
   - Use realistic values that make sense together
   - Follow the correlations and dependencies described
   - Ensure the combination is internally consistent
   - Use appropriate types (numbers for numeric attributes, strings for categories, booleans for yes/no)

2. profile_summary: Write a 2-3 sentence natural language description of this entity
   that captures who they are in a human-readable way.

Example of good profile_summary:
"A 34-year-old software engineer living in Seattle with high income and low price sensitivity.
Has been using the product for 18 months and considers it essential to their workflow.
Values quality over cost and has few alternatives available."

Be creative but realistic. Make this feel like a real person/entity."""

    result = await app.ai(prompt, schema=EntityProfile)

    result.entity_id = entity_id
    return result

In [32]:
async def generate_entity_batch(
    start_id: int,
    batch_size: int,
    scenario_analysis: ScenarioAnalysis,
    factor_graph: FactorGraph,
    exploration_ratio: float = 0.1,
) -> List[EntityProfile]:
    """
    Generate a batch of entities in parallel.
    """
    tasks = []
    for i in range(batch_size):
        entity_id = f"E_{start_id + i:06d}"
        # Decide if this should be exploration or standard
        exploration_mode = i < int(batch_size * exploration_ratio)

        task = generate_entity_profile(
            entity_id, scenario_analysis, factor_graph, exploration_mode
        )
        tasks.append(task)

    entities = await asyncio.gather(*tasks)
    return entities

In [33]:
class EntityDecision(BaseModel):
    """Schema for entity's decision"""

    entity_id: str
    decision: str = Field(
        description="The chosen decision/action from the available options"
    )
    confidence: float = Field(description="Confidence in this decision, 0.0 to 1.0")
    reasoning: str = Field(
        description="Detailed explanation (3-4 sentences) of why this entity made this decision, referencing their specific attributes and the scenario"
    )


async def simulate_entity_decision(
    entity: EntityProfile,
    scenario: str,
    scenario_analysis: ScenarioAnalysis,
    context: List[str] = [],
) -> EntityDecision:
    """
    Simulates what decision this specific entity would make.
    """
    context_str = "\n".join([f"- {c}" for c in context]) if context else ""

    # Format attributes in a readable way
    attributes_str = "\n".join([f"  • {k}: {v}" for k, v in entity.attributes.items()])

    # Build context section separately to avoid backslash in f-string expression
    context_section = f"ADDITIONAL CONTEXT:\n{context_str}\n\n" if context else ""

    prompt = f"""You are simulating the decision-making of a specific {scenario_analysis.entity_type}.

WHO YOU ARE:
{entity.profile_summary}

YOUR ATTRIBUTES:
{attributes_str}

SCENARIO YOU'RE FACING:
{scenario}

{context_section}AVAILABLE DECISIONS:
{', '.join(scenario_analysis.decision_options)}

TASK:
Based on who you are (your specific attributes above), decide how you would respond to this scenario.

1. decision: Choose one option from the available decisions list. Pick the one that makes most sense given your attributes and situation.

2. confidence: Rate your confidence in this decision from 0.0 (very uncertain) to 1.0 (completely certain).
   Consider:
   - How clear is your preference?
   - How much are you torn between options?
   - How much information do you have?

3. reasoning: Explain your decision in 3-4 sentences. Specifically reference:
   - Which of your attributes influenced this decision most
   - What trade-offs you considered
   - Why you chose this over alternatives
   - Any uncertainty or caveats

Think step-by-step:
- What aspects of your profile are most relevant here?
- How do your specific attributes (age, income, tenure, etc.) affect your thinking?
- What would someone like you typically do?
- Are there any unique factors in your situation?

Be realistic and consistent with your attributes. Make this feel like a real decision by a real {scenario_analysis.entity_type}."""

    result = await app.ai(prompt, schema=EntityDecision)

    result.entity_id = entity.entity_id
    return result

In [34]:
async def simulate_batch_decisions(
    entities: List[EntityProfile],
    scenario: str,
    scenario_analysis: ScenarioAnalysis,
    context: List[str] = [],
) -> List[EntityDecision]:
    """
    Simulate decisions for a batch of entities in parallel.
    """
    tasks = [
        simulate_entity_decision(entity, scenario, scenario_analysis, context)
        for entity in entities
    ]

    decisions = await asyncio.gather(*tasks)
    return decisions

In [35]:
class SimulationInsights(BaseModel):
    """Schema for final simulation results"""

    outcome_distribution: Dict[str, float] = Field(
        description="Percentage for each decision option"
    )
    key_insight: str = Field(
        description="One sentence summary of the most important finding"
    )
    detailed_analysis: str = Field(
        description="Comprehensive analysis (4-5 paragraphs) covering: overall patterns, segment differences, causal drivers, surprising findings, and implications"
    )
    segment_patterns: str = Field(
        description="Description of how different types of entities decided differently, organized by meaningful segments"
    )
    causal_drivers: str = Field(
        description="Analysis of which attributes most strongly predicted decisions, with specific examples and correlations"
    )


async def aggregate_and_analyze(
    scenario: str,
    scenario_analysis: ScenarioAnalysis,
    factor_graph: FactorGraph,
    entities: List[EntityProfile],
    decisions: List[EntityDecision],
    context: List[str] = [],
) -> SimulationInsights:
    """
    Aggregates all decisions and generates insights.
    """
    # Compute basic statistics
    total = len(decisions)
    decision_counts = {}
    for d in decisions:
        decision_counts[d.decision] = decision_counts.get(d.decision, 0) + 1

    outcome_dist = {k: v / total for k, v in decision_counts.items()}

    # Prepare data summary for AI
    context_str = "\n".join([f"- {c}" for c in context]) if context else ""

    # Sample entities and decisions for analysis (to avoid token limits)
    sample_size = min(50, len(entities))
    sample_indices = list(
        range(0, len(entities), max(1, len(entities) // sample_size))
    )[:sample_size]

    samples = []
    for i in sample_indices:
        entity = entities[i]
        decision = decisions[i]
        samples.append(
            {
                "attributes": entity.attributes,
                "decision": decision.decision,
                "reasoning": decision.reasoning,
                "confidence": decision.confidence,
            }
        )

    samples_str = json.dumps(samples, indent=2)

    # Avoid f-string inside f-string; fully construct context if needed
    if context:
        context_block = "CONTEXT:\n" + context_str + "\n"
    else:
        context_block = ""

    scenario_entity_type = scenario_analysis.entity_type
    attributes_list = list(factor_graph.attributes.keys())
    outcome_dist_json = json.dumps(outcome_dist, indent=2)

    prompt = (
        f"You are analyzing the results of a simulation with {total} entities.\n\n"
        f"SCENARIO SIMULATED:\n"
        f"{scenario}\n\n"
        f"{context_block}"
        f"ENTITY TYPE: {scenario_entity_type}\n"
        f"ATTRIBUTES TRACKED: {attributes_list}\n\n"
        f"OUTCOME DISTRIBUTION:\n"
        f"{outcome_dist_json}\n\n"
        f"SAMPLE OF ENTITIES AND THEIR DECISIONS:\n"
        f"{samples_str}\n\n"
        "TASK:\n"
        "Analyze these simulation results deeply and provide insights:\n\n"
        "1. outcome_distribution: Already provided above - just return this exact dictionary: "
        f"{outcome_dist}\n\n"
        "2. key_insight: Write ONE sentence that captures the most important or surprising finding from this simulation.\n\n"
        "3. detailed_analysis: Write 4-5 paragraphs covering:\n"
        "   - What is the overall pattern? What's the dominant outcome and why?\n"
        "   - Are there distinct segments that behaved differently? Describe them.\n"
        "   - What are the key drivers? Which attributes most strongly predicted decisions?\n"
        "   - Were there any surprising or counterintuitive findings?\n"
        "   - What are the implications? What should someone do with this information?\n\n"
        "4. segment_patterns: Analyze how different types of entities decided differently.\n"
        "   - Group entities by meaningful combinations of attributes\n"
        "   - Describe each segment's typical decision and why\n"
        "   - Note which segments are most/least certain\n"
        "   - Identify any interesting edge cases\n"
        "   (Write 2-3 paragraphs)\n\n"
        "5. causal_drivers: Identify which attributes most strongly influenced decisions.\n"
        "   - For each major attribute, describe its effect on decisions\n"
        "   - Provide specific examples from the data\n"
        '   - Describe interaction effects (e.g., "age matters more for low-income entities")\n'
        "   - Rank drivers by importance\n"
        "   (Write 2-3 paragraphs)\n\n"
        "Be specific, reference the data, and provide actionable insights."
    )

    result = await app.ai(prompt, schema=SimulationInsights)

    # Override outcome_distribution with our computed values
    result.outcome_distribution = outcome_dist

    return result

In [36]:
scenario = """
Our SaaS product currently has a free tier that 50% of our users are on.
We're considering removing the free tier and offering a $29/month starter plan instead.
How will existing free users react?
"""

context = [
    "Average free user has been with us 8 months",
    "Current paid conversion rate from free is 3%",
    "Two main competitors (CompetitorA and CompetitorB) offer free tiers",
    "Our product is primarily used by small businesses and freelancers",
]

# result = await run_simulation(
#     app=app,
#     scenario=scenario,
#     population_size=1000,  # Start with 1K for testing
#     context=context,
#     parallel_batch_size=50,
#     exploration_ratio=0.1
# )

In [37]:
# Step 1: Decompose the scenario
scenario_analysis = await decompose_scenario(scenario, context)

In [38]:
# Display the scenario analysis
print("Scenario Analysis:")
print(f"Entity Type: {scenario_analysis.entity_type}")
print(f"Decision Type: {scenario_analysis.decision_type}")
print(f"Decision Options: {scenario_analysis.decision_options}")
print(f"\nAnalysis:\n{scenario_analysis.analysis}")

Scenario Analysis:
Entity Type: customer
Decision Type: multi_option
Decision Options: ['Upgrade to the $29/month starter plan', 'Downgrade usage to stay within limits of a hypothetical new free tier (if any remain)', 'Churn and switch to CompetitorA', 'Churn and switch to CompetitorB', 'Churn and switch to a different solution not mentioned', 'Churn and stop using this type of product entirely', 'Attempt to negotiate or seek a discount', 'Remain inactive on a free plan (if grandfathered)']

Analysis:
The decision of existing free users is influenced by several key factors. Financial considerations are paramount - small businesses and freelancers have varying budget constraints and price sensitivity. Product dependency matters, as users who have integrated our tool into their workflows face higher switching costs. Perceived value is crucial; users must feel the starter plan offers sufficient benefit over the free tier. Competitive alternatives from CompetitorA and CompetitorB offering 

In [39]:
# Step 2: Generate factor graph
factor_graph = await generate_factor_graph(scenario, scenario_analysis, context)

In [40]:
# Display the factor graph
print("Factor Graph Attributes:")
print(json.dumps(factor_graph.attributes, indent=2))
print("\nAttribute Graph:")
print(factor_graph.attribute_graph)
print("\nSampling Strategy:")
print(factor_graph.sampling_strategy)

Factor Graph Attributes:
{
  "age": "Age of the primary user or business decision-maker",
  "annual_revenue": "Annual revenue of the user's business, a key indicator of budget capacity",
  "business_type": "Type of business (e.g., solopreneur, small_team, freelance)",
  "competitor_awareness": "Awareness level of competitor offerings, especially free tiers from CompetitorA and CompetitorB",
  "contract_status": "Whether the user is under any contractual obligations that affect switching costs",
  "feature_dependency": "Degree to which the user relies on specific, potentially paid-only, features",
  "industry": "Industry in which the user's business operates",
  "loss_aversion": "Psychological tendency to prefer avoiding losses over acquiring equivalent gains",
  "monthly_budget": "Monthly budget allocated for software tools",
  "perceived_value": "User's subjective assessment of the product's worth relative to cost",
  "price_sensitivity": "Degree to which the user's decision is influe

In [41]:
# Step 3: Generate a small batch of entities (for testing)
# Adjust batch_size as needed
batch_size = 10
entities = await generate_entity_batch(
    start_id=0,
    batch_size=batch_size,
    scenario_analysis=scenario_analysis,
    factor_graph=factor_graph,
    exploration_ratio=0.1,
)

print(f"Generated {len(entities)} entities")

Generated 10 entities


In [42]:
# Display a sample entity to inspect
print("Sample Entity:")
sample_entity = entities[0]
print(f"Entity ID: {sample_entity.entity_id}")
print(f"Profile Summary: {sample_entity.profile_summary}")
print("\nAttributes:")
print(json.dumps(sample_entity.attributes, indent=2))

Sample Entity:
Entity ID: E_000000
Profile Summary: Algorand Labs is a high-revenue fintech firm deeply dependent on our product for minute-by-minute trading operations, making switching costs prohibitive. Despite their substantial budget and low tenure, the firm's leadership exhibits extreme price sensitivity and loss aversion, constantly evaluating competitor offerings. This creates a volatile customer profile where the immense workflow integration conflicts with a relentless focus on cost containment.

Attributes:
{
  "age": 52,
  "annual_revenue": 4200000,
  "business_type": "small_team",
  "competitor_awareness": "high",
  "contract_status": "none",
  "feature_dependency": "critical",
  "industry": "fintech",
  "loss_aversion": "very_high",
  "monthly_budget": 75000,
  "perceived_value": "fair",
  "price_sensitivity": "very_high",
  "product_tenure": 3,
  "region": "New York",
  "switching_costs": "prohibitive",
  "usage_frequency": "minutely",
  "workflow_integration": "total"
}


In [43]:
# Step 4: Simulate decisions for all entities
decisions = await simulate_batch_decisions(
    entities=entities,
    scenario=scenario,
    scenario_analysis=scenario_analysis,
    context=context,
)

print(f"Simulated {len(decisions)} decisions")

Simulated 10 decisions


In [44]:
# Display a sample decision to inspect
print("Sample Decision:")
sample_decision = decisions[0]
print(f"Entity ID: {sample_decision.entity_id}")
print(f"Decision: {sample_decision.decision}")
print(f"Confidence: {sample_decision.confidence}")
print(f"Reasoning: {sample_decision.reasoning}")

Sample Decision:
Entity ID: E_000000
Decision: Attempt to negotiate or seek a discount
Confidence: 0.85
Reasoning: My very high price sensitivity and loss aversion make me immediately resistant to paying $29 monthly for a product I've used for free, despite its critical feature dependency and prohibitive switching costs which make outright churning impractical. I will attempt to negotiate a deeply discounted starter plan, leveraging my firm's high revenue and the threat of evaluating CompetitorA or CompetitorB, who still offer free tiers, to secure a better deal. This approach balances my need for cost containment with the reality that my minutely trading operations depend entirely on your product's total workflow integration. However, my low tenure and the fact I only perceive the product's value as fair introduces uncertainty about the success of these negotiations.


In [45]:
# Quick summary of decisions
decision_counts = {}
for d in decisions:
    decision_counts[d.decision] = decision_counts.get(d.decision, 0) + 1

print("Decision Distribution:")
for decision, count in decision_counts.items():
    percentage = (count / len(decisions)) * 100
    print(f"  {decision}: {count} ({percentage:.1f}%)")

Decision Distribution:
  Attempt to negotiate or seek a discount: 1 (10.0%)
  Upgrade to the $29/month starter plan: 1 (10.0%)
  Churn and switch to CompetitorA: 8 (80.0%)


In [46]:
# Step 5: Aggregate and analyze results
insights = await aggregate_and_analyze(
    scenario=scenario,
    scenario_analysis=scenario_analysis,
    factor_graph=factor_graph,
    entities=entities,
    decisions=decisions,
    context=context,
)

In [47]:
# Display the insights
print("=" * 60)
print("KEY INSIGHT:")
print("=" * 60)
print(insights.key_insight)

print("\n" + "=" * 60)
print("OUTCOME DISTRIBUTION:")
print("=" * 60)
for decision, pct in insights.outcome_distribution.items():
    print(f"  {decision}: {pct*100:.1f}%")

print("\n" + "=" * 60)
print("DETAILED ANALYSIS:")
print("=" * 60)
print(insights.detailed_analysis)

print("\n" + "=" * 60)
print("SEGMENT PATTERNS:")
print("=" * 60)
print(insights.segment_patterns)

print("\n" + "=" * 60)
print("CAUSAL DRIVERS:")
print("=" * 60)
print(insights.causal_drivers)

KEY INSIGHT:
The overwhelming 80% churn rate reveals that most free users value price certainty over product features, with even long-term, moderately satisfied users abandoning the platform when faced with sudden paid requirements.

OUTCOME DISTRIBUTION:
  Attempt to negotiate or seek a discount: 10.0%
  Upgrade to the $29/month starter plan: 10.0%
  Churn and switch to CompetitorA: 80.0%

DETAILED ANALYSIS:
The simulation reveals a stark outcome where 80% of free users would churn to CompetitorA upon removal of the free tier, indicating extreme vulnerability in the user base. This overwhelming churn rate stems from the combination of high price sensitivity among freelancers and solopreneurs, widespread awareness of competing free alternatives, and generally low switching costs for most users. Only 10% would upgrade to the paid plan, while another 10% would attempt negotiation, suggesting the current $29 pricing matches the willingness-to-pay of only a small, financially secure segmen